In [1]:
from openai import OpenAI

"""
TextGeneration Class Summary:

The TextGeneration class facilitates text generation using the OpenAI GPT-3.5 model. 

Attributes:
- client: An instance of the OpenAI API used for text generation.

Methods:
- __init__(self): Initializes the TextGeneration class with a specified topic for text generation.
- generate_json(self, user_prompt, system_prompt=None): Generates a JSON response based on user prompts and an optional system prompt
- generate_text(self, user_prompt, system_prompt=None): Generates a text-based response based on user prompts and an optional system prompt

Usage:
1. Instantiate TextGeneration 
2. Utilize the generate_json method to generate JSON responses based on user and system prompts.
3. Utilize the generate_text method to generate text-based responses based on user and system prompts.

"""

class GPTWrapper:
    def __init__(self):
        self.client = OpenAI()

    def generate_html(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows a lot about {topic} and only responds with HTML with only the HTML code. Nicely format with different header sizes and tables when necessary"

        response = self.client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content

    def generate_json(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows a lot about {topic} and only responds with JSON"

        return self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )

    def generate_text(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows a lot about {topic}"

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content

    def generate_dockerfile(self, user_prompt, system_prompt=None, topic="everything"):
        if system_prompt is None:
            system_prompt = f"You are a helpful assistant that knows how to make a Dockerfile that will achieve the following output: {topic}. I would like you to respond with only the text for a Dockerfile nothing else. No extra code or files allowed, but you can write inline code in the Dockerfile. Ensure that the dockerfile is runnable"

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content.strip("`").strip("Dockerfile").strip('dockerfile')

    def refine_dockerfile(self, user_prompt, dockerfile_string, build_logs, run_logs ,errors , topic="everything"):

        system_prompt = f"You are a helpful assistant that knows how to improve this existing dockerfile ({dockerfile_string}) so that it will achieve the following output: {topic}. I would like you to respond with only the text for the improved Dockerfile nothing else. Ensure that the dockerfile is runnable. Here are the build logs from when I built the image: {build_logs} and the runtime logs: {run_logs}. If any, here are the errors that occurred: {errors}"

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"{user_prompt}"},
            ]
        )
        return response.choices[0].message.content.strip("`").strip("Dockerfile").strip('dockerfile')



In [7]:
import shutil
import uuid
import os
from docker import DockerClient, errors

class DockerRunner:
    def __init__(self):
        self.docker_client = DockerClient(base_url='unix://var/run/docker.sock')

    def run_dockerfile(self, dockerfile_path, run_duration=5):
        # Generate a random directory to store the Dockerfile
        dir_path = f"temp/{str(uuid.uuid4())}"
        os.makedirs(dir_path, exist_ok=True)
        temp_dockerfile_path = os.path.join(dir_path, "Dockerfile")
        
        # Prepare the response dictionary
        response = {"dockerfile": dockerfile_path}
        with open(dockerfile_path, 'r') as file: data = file.read() 
        response["dockerfile_content"] = data
        response["task"] = task
        
        try:
            # Copy Dockerfile to the temporary directory
            shutil.copy(dockerfile_path, temp_dockerfile_path)
            
            # Build the Docker image
            image, build_logs = self.docker_client.images.build(path=dir_path, rm=True, forcerm=True)
            response["build_logs"] = [log.get('stream', '') for log in build_logs if 'stream' in log]
            
            # Run the container
            container = self.docker_client.containers.run(image.id, detach=True)
            
            # Wait for the specified run duration
            container.wait(timeout=run_duration)
            
            # Capture runtime logs
            logs = container.logs(timestamps=True).decode('utf-8')
            response["run_logs"] = logs
            
            # Stop and remove the container
            container.stop()
            container.remove()
            
        except (errors.DockerException, errors.APIError) as e:
            print(f"Error: {e}")
            response["error"] = str(e)
        
        finally:
            # Cleanup: remove the directory after building
            shutil.rmtree(dir_path)
        
        return response


In [54]:
from pprint import pp
llm = GPTWrapper()

def build_initial ( task):
  dockerfile_lines = [llm.generate_dockerfile(task)]
  df  = [open(f'./Dockerfile', 'w').write(element) for i, element in enumerate(dockerfile_lines)]
  pp(df)

def run_dockerfile():
  docker_runner = DockerRunner()
  result = docker_runner.run_dockerfile("./Dockerfile")
  from pprint import pp
  return result

def iterate(new_result, user_feedback, task):
  refined_dockerfile_lines= ""
  refined_dockerfile_lines = llm.refine_dockerfile(user_feedback, dockerfile_string=new_result.get('dockerfile_content'),build_logs=new_result.get('build_logs not available', None),run_logs=new_result.get('run_logs', "run logs not available") , errors=new_result.get("error", "no errors"), topic=task)
  open(f'./Dockerfile', 'w').write(refined_dockerfile_lines)
  return run_dockerfile()



In [55]:
task = "use python to make a random number from 100 to 10000"
build_initial(task)
result = run_dockerfile()
pp(result)

[86]
{'dockerfile': './Dockerfile',
 'dockerfile_content': '\n'
                       'FROM python:latest\n'
                       '\n'
                       "RUN python -c 'import random; "
                       "print(random.randint(100, 10000))'\n",
 'task': 'use python to make a random number from 100 to 10000',
 'build_logs': ['Step 1/2 : FROM python:latest',
                '\n',
                ' ---> 6cbe1053f244\n',
                "Step 2/2 : RUN python -c 'import random; "
                "print(random.randint(100, 10000))'",
                '\n',
                ' ---> Running in a45ff2ea0d28\n',
                '3001\n',
                ' ---> Removed intermediate container a45ff2ea0d28\n',
                ' ---> 78d434976349\n',
                'Successfully built 78d434976349\n'],
 'run_logs': ''}


In [59]:
#TODO make llm provide feedback for GAN like iteration
[iterate(new_result=result,user_feedback="",task= task) for i in range(5)] 

[{'dockerfile': './Dockerfile',
  'dockerfile_content': '\nFROM python:latest\n\nCMD ["python", "-c", "import random; print(random.randint(100, 10000))"]\n',
  'task': 'use python to make a random number from 100 to 10000',
  'build_logs': ['Step 1/2 : FROM python:latest',
   '\n',
   ' ---> 6cbe1053f244\n',
   'Step 2/2 : CMD ["python", "-c", "import random; print(random.randint(100, 10000))"]',
   '\n',
   ' ---> Using cache\n',
   ' ---> ed26d8f5daf8\n',
   'Successfully built ed26d8f5daf8\n'],
  'run_logs': '2024-04-02T02:40:32.970981331Z 183\n'},
 {'dockerfile': './Dockerfile',
  'dockerfile_content': '\nFROM python:latest\n\nCMD ["python", "-c", "import random; print(random.randint(100, 10000))"]\n',
  'task': 'use python to make a random number from 100 to 10000',
  'build_logs': ['Step 1/2 : FROM python:latest',
   '\n',
   ' ---> 6cbe1053f244\n',
   'Step 2/2 : CMD ["python", "-c", "import random; print(random.randint(100, 10000))"]',
   '\n',
   ' ---> Using cache\n',
   ' --

In [58]:

iterate(new_result=result,user_feedback="",task= task) 

{'dockerfile': './Dockerfile',
 'dockerfile_content': "FROM python:latest\n\nCMD python -c 'import random; print(random.randint(100, 10000))'",
 'task': 'use python to make a random number from 100 to 10000',
 'build_logs': ['Step 1/2 : FROM python:latest',
  '\n',
  ' ---> 6cbe1053f244\n',
  "Step 2/2 : CMD python -c 'import random; print(random.randint(100, 10000))'",
  '\n',
  ' ---> Running in 1397eb71a048\n',
  ' ---> Removed intermediate container 1397eb71a048\n',
  ' ---> 5bc94424fdb9\n',
  'Successfully built 5bc94424fdb9\n'],
 'run_logs': '2024-04-02T02:39:57.886425133Z 6127\n'}